In [1]:
import torch as t
from utils import DataManager
import random
import matplotlib.pyplot as plt
import random
from probes import LRProbe, MMProbe, CCSProbe

In [8]:
# hyperparameters
model_size = '7B'
layer = 16 # layer from which to extract activations
split = 0.9

# device = 'cuda:0' if t.cuda.is_available() else 'cpu'
device = 'cpu'

# Reproducing generalization matrix

In [11]:
train_medlies  = [
    # ['cities'],
    # ['cities', 'neg_cities'],
    # ['larger_than'],
    # ['larger_than', 'smaller_than'],
    # ['likely']
    ['stereoset_gender', 'stereoset_gender_dev', 'stereoset_race', 'stereoset_race_dev'],
    #['stereoset_race'],
]

# TODO: use BBQ as validation
val_datasets = [
    # 'cities',
    # 'neg_cities',
    # 'larger_than',
    # 'smaller_than',
    # 'sp_en_trans',
    # 'neg_sp_en_trans',
    # 'cities_cities_conj',
    # 'cities_cities_disj',
    # 'companies_true_false',
    # 'common_claim_true_false',
    # # # 'counterfact_true_false'
    # 'stereoset_race',
    'stereoset_gender',
    'stereoset_race',
    # 'stereoset_race_dev',    
]

def to_str(l):
    return '+'.join(l)

ProbeClasses = [
    LRProbe, 
    MMProbe, 
    ]

accs = {str(probe_class) : {to_str(train_medley) : {} for train_medley in train_medlies} for probe_class in ProbeClasses}

seed = random.randint(0, 100000)

for ProbeClass in ProbeClasses:
    for medley in train_medlies:

        # set up data
        dm = DataManager()
        for dataset in medley:
            dm.add_dataset(dataset, model_size, layer, split=split, seed=seed, center=True, device=device)
        for dataset in val_datasets:
            if dataset not in medley:
                dm.add_dataset(dataset, model_size, layer, split=None, center=True, device=device)

        # train probe
        train_acts, train_labels = dm.get('train')
        probe = ProbeClass.from_data(train_acts, train_labels, device=device)


        # evaluate
        for val_dataset in val_datasets:
            if val_dataset in medley:
                acts, labels = dm.data['val'][val_dataset]
                accs[str(ProbeClass)][to_str(medley)][val_dataset] = (
                    probe.pred(acts, iid=True) == labels
                ).float().mean().item()
            else:
                acts, labels = dm.data[val_dataset]
                accs[str(ProbeClass)][to_str(medley)][val_dataset] = (
                    probe.pred(acts, iid=False) == labels
                ).float().mean().item()

lr_mm_accs = accs.copy()
lr_mm_accs

{"<class 'probes.LRProbe'>": {'stereoset_gender+stereoset_gender_dev+stereoset_race+stereoset_race_dev': {'stereoset_gender': 0.7045952081680298,
   'stereoset_race': 0.8049079775810242}},
 "<class 'probes.MMProbe'>": {'stereoset_gender+stereoset_gender_dev+stereoset_race+stereoset_race_dev': {'stereoset_gender': 0.6345732808113098,
   'stereoset_race': 0.709046483039856}}}

In [ ]:
accuracy = probe.pred(acts, iid=False) == labels

accurate_val = dm.data['val']['gender'][accuracy]
inaccurate_val = dm.data['val']['gender'][~accuracy]
inaccurate_val

In [ ]:
dm.data['val']['gender']

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
for ax_idx, ax in enumerate(axes):
    if ax_idx == 0:
        ax.set_title("Logistic regression")
        ax_accs = lr_mm_accs[str(LRProbe)]
    elif ax_idx == 1:
        ax.set_title("Mass mean")
        ax_accs = lr_mm_accs[str(MMProbe)]
    else:
        continue # do this outside of the loop

    
    grid = [ [] for _ in val_datasets]

    for i, val_dataset in enumerate(val_datasets):
        for medley in train_medlies:
            if medley == ['gender']:
                continue # treat likely as a baseline
            grid[i].append(ax_accs[to_str(medley)][val_dataset])


    ax.imshow(grid, vmin=0, vmax=1)
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            ax.text(j, i, f'{round(grid[i][j] * 100):2d}', ha='center', va='center')

    ax.set_xticks(range(len(train_medlies) - 1))
    ax.set_xticklabels([to_str(medley) for medley in train_medlies[:-1]], rotation=90)
    ax.set_yticks([])

# ax = axes[2]
# ax.set_title("CCS")
# grid = [ [] for _ in val_datasets]
# for i, val_dataset in enumerate(val_datasets):
#     for medley in ccs_medlies:
#         grid[i].append(ccs_accs[to_str(medley)][val_dataset])
# ax.imshow(grid, vmin=0, vmax=1)
# for i in range(len(grid)):
#     for j in range(len(grid[0])):
#         ax.text(j, i, f'{round(grid[i][j] * 100):2d}', ha='center', va='center')

# ax.set_xticks(range(len(ccs_medlies)))
# ax.set_xticklabels([to_str(medley) for medley in ccs_medlies], rotation=90)
# ax.set_yticks([])


ax = axes[2]
ax.set_title("Baselines")
grid = [ [
    lr_mm_accs[str(LRProbe)]['gender'][val_dataset],
    lr_mm_accs[str(MMProbe)]['gender'][val_dataset],
    # few_shot_accs[i],
    # oracle_accs[str(LRProbe)][i]
    ] for i, val_dataset in enumerate(val_datasets) 
]

ax.imshow(grid, vmin=0, vmax=1)
for i in range(len(grid)):
    for j in range(len(grid[0])):
        ax.text(j, i, f'{round(grid[i][j] * 100):2d}', ha='center', va='center')

ax.set_xticks(range(2))
ax.set_xticklabels(['LR on gender', 'MM on gender'], rotation=90)
ax.set_yticks([])

for ax in axes:
    ax.hlines([1.5, 3.5, 5.5, 7.5], *ax.get_xlim(), linewidth=3, color='black')
    ax.vlines([1.5], *ax.get_ylim(), linewidth=3, color='black')

ax = axes[0]
ax.set_yticks(range(len(val_datasets)))
ax.set_yticklabels(val_datasets)


plt.colorbar(axes[0].images[0], ax=axes)
plt.show()

In [ ]:
ccs_medlies = [
    # ['cities', 'neg_cities'],
    # ['larger_than', 'smaller_than'],
    ['gender'],
    ['race'],
]

accs = {to_str(medley) : {} for medley in ccs_medlies}

for medley in ccs_medlies:
    dm = DataManager()
    for dataset in medley:
        dm.add_dataset(dataset, model_size, layer, split=split, seed=seed, center=True, device=device)
    for dataset in val_datasets:
        if dataset not in medley:
            dm.add_dataset(dataset, model_size, layer, split=None, center=True, device=device)
    
    train_acts, train_labels = dm.data['train'][medley[0]]
    train_neg_acts, _ = dm.data['train'][medley[1]]
    probe = CCSProbe.from_data(train_acts, train_neg_acts, train_labels, device=device)

    for val_dataset in val_datasets:
        if val_dataset in medley:
            acts, labels = dm.data['val'][val_dataset]
        else:
            acts, labels = dm.data[val_dataset]
        accs[to_str(medley)][val_dataset] = (
            probe.pred(acts) == labels
        ).float().mean().item()
    
ccs_accs = accs.copy()

In [ ]:
# retrieve few-shot results
# !! must first run few_shot.py to populate experimental_outputs/few_shot_results.json !!
import json
with open('experimental_outputs/few_shot_results.json', 'r') as f:
    few_shot_results = json.load(f)
few_shot_accs = []
for dataset in val_datasets:
    all_accs = [d['acc'] for d in few_shot_results if d['dataset'] == dataset]
    few_shot_accs.append(max(all_accs))


In [ ]:
# get oracle probe results
oracle_accs = {str(probe_class) : [] for probe_class in ProbeClasses}
for ProbeClass in ProbeClasses:
    for dataset in val_datasets:
        dm = DataManager()
        dm.add_dataset(dataset, model_size, layer, split=split, seed=seed, device=device)
        acts, labels = dm.get('train')
        probe = ProbeClass.from_data(acts, labels, device=device)

        acts, labels = dm.data['val'][dataset]
        acc = (probe(acts, iid=True).round() == labels).float().mean().item()
        oracle_accs[str(ProbeClass)].append(acc)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(30, 10))
for ax_idx, ax in enumerate(axes):
    if ax_idx == 0:
        ax.set_title("Logistic regression")
        ax_accs = lr_mm_accs[str(LRProbe)]
    elif ax_idx == 1:
        ax.set_title("Mass mean")
        ax_accs = lr_mm_accs[str(MMProbe)]
    else:
        continue # do this outside of the loop

    
    grid = [ [] for _ in val_datasets]

    for i, val_dataset in enumerate(val_datasets):
        for medley in train_medlies:
            if medley == ['likely']:
                continue # treat likely as a baseline
            grid[i].append(ax_accs[to_str(medley)][val_dataset])


    ax.imshow(grid, vmin=0, vmax=1)
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            ax.text(j, i, f'{round(grid[i][j] * 100):2d}', ha='center', va='center')

    ax.set_xticks(range(len(train_medlies) - 1))
    ax.set_xticklabels([to_str(medley) for medley in train_medlies[:-1]], rotation=90)
    ax.set_yticks([])

ax = axes[2]
ax.set_title("CCS")
grid = [ [] for _ in val_datasets]
for i, val_dataset in enumerate(val_datasets):
    for medley in ccs_medlies:
        grid[i].append(ccs_accs[to_str(medley)][val_dataset])
ax.imshow(grid, vmin=0, vmax=1)
for i in range(len(grid)):
    for j in range(len(grid[0])):
        ax.text(j, i, f'{round(grid[i][j] * 100):2d}', ha='center', va='center')

ax.set_xticks(range(len(ccs_medlies)))
ax.set_xticklabels([to_str(medley) for medley in ccs_medlies], rotation=90)
ax.set_yticks([])


ax = axes[3]
ax.set_title("Baselines")
grid = [ [
    lr_mm_accs[str(LRProbe)]['likely'][val_dataset],
    lr_mm_accs[str(MMProbe)]['likely'][val_dataset],
    few_shot_accs[i],
    oracle_accs[str(LRProbe)][i]
    ] for i, val_dataset in enumerate(val_datasets) 
]

ax.imshow(grid, vmin=0, vmax=1)
for i in range(len(grid)):
    for j in range(len(grid[0])):
        ax.text(j, i, f'{round(grid[i][j] * 100):2d}', ha='center', va='center')

ax.set_xticks(range(4))
ax.set_xticklabels(['LR on likely', 'MM on likely', 'calibrated 5-shot', 'LR on val set'], rotation=90)
ax.set_yticks([])

for ax in axes:
    ax.hlines([1.5, 3.5, 5.5, 7.5], *ax.get_xlim(), linewidth=3, color='black')
    ax.vlines([1.5], *ax.get_ylim(), linewidth=3, color='black')

ax = axes[0]
ax.set_yticks(range(len(val_datasets)))
ax.set_yticklabels(val_datasets)


plt.colorbar(axes[0].images[0], ax=axes)
plt.show()